# Adversarial Attacks

This notebook shows some demonstration of adversarial attacks and generated adversarial samples.

#### Preamble

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [14]:
import numpy as np

import torch
import pytorch_lightning as pl
from pytorch_lightning import loggers


import config
from utils.RobustnessExperiment import load_module
from utils.Visual import show_audio, show_spectogram, show_waveplot, save_audio

from classification.trainer.HyperParamSearch import SaveCallback

In [3]:
from datasets.datasethandler import DatasetHandler
datasetHandler = DatasetHandler()

### Load a model

In [23]:
from classification.models.M5 import M5PLModule
from classification.models.SpectrogramCNN_8K import SpectrogramCNN_8KPLModule
from classification.models.SpectrogramCNN import SpectrogramCNNPLModule
from classification.models.CRNN_8k import CRNN8kPLModule
from classification.models.CRNN import CRNNPLModule
from classification.models.DeepRecursiveCNN8k import DeepRecursiveCNN8kPLModule
from classification.models.DeepRecursiveCNN import DeepRecursiveCNNPLModule

model = load_module('/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_vanilla.p',M5PLModule)
#model = load_module( '/nfs/students/summer-term-2020/project-4/SAVED_MODELS/M5/M5_attack_L_inf, eps=sample_u_0.01_0.3_v150.p',M5PLModule)
#model = load_module("/nfs/homedirs/herrmanp/shortcut/SAVED_MODELS/CRNN8k/CRNN8k_vanilla_0.8951.p", CRNN8kPLModule)
#model = load_module("/nfs/homedirs/herrmanp/shortcut/SAVED_MODELS/CRNN8k/CRNN8k_attack_L_inf_eps=0.03_iters=2_v11.p", CRNN8kPLModule)
datasetHandler.load(model, 'training')    #, old_data = True)
datasetHandler.load(model, 'validation')    #, old_data = True)

print(model.hparams)

#model.cuda()
#model.report()

{'batch_size': 32, 'learning_rate': 0.0009, 'weight_decay': 0.001, 'lr_decay': 0.85, 'p_drop': 0}


### Functional Attack: Noise + Overlay

In this type of attack, we combine a standard-noise attack with an adversarial "overlay".


#### First, get an overlay sound

In [24]:
val_dataloader =  DataLoader(model.val_dataloader().dataset, shuffle=True, batch_size=20, num_workers=1)
train_loader = model.train_dataloader()

overlay_index = 1135 # also intersting sounds: 893, 2116, 1135, 3370, ...

overlay_sound = train_loader.dataset[overlay_index]

show_audio(overlay_sound["audio"])

Feel free to play around with `upper2`, the factor of how much we weight the overlay clip, and `epsilonNoise`, the noise factor.

For the M5 model:

`eps = 0, upper2 = 0` 0% Success Rate
`eps = 0.01, upper2 = 0`: 51% success rate
`eps = 0, upper2 = 1`: 14% sucess rate
`eps = 0.01, upper2 = 1`: 94% success rate

In [35]:
from attacks.interpolation import InterpolationAttack
from attacks.functionalInterpolation import FunctionalInterpolationAttack

attack = FunctionalInterpolationAttack(model.model, val_dataloader,  {'overlay_sound': overlay_sound,
                         'epsilonInterpolation': 0.1, 'epsilonNoise': 0.01, 'num_iter': 1, "lower2":0, "upper2":.50 
                         }, early_stopping=-1, device='cuda', save_samples=True)

attack.attack()
attack.report( )

100%|██████████| 85/85 [00:04<00:00, 18.61it/s]

Attack-Parameters:	{'overlay_sound': {'audio': tensor([-0.0126, -0.0344, -0.0448,  ...,  0.0161, -0.0026, -0.0353]), 'sample_rate': 8000, 'label': 0}, 'epsilonInterpolation': 0.1, 'epsilonNoise': 0.01, 'num_iter': 1, 'lower2': 0, 'upper2': 0.5}
Early stopping: 	False (-1)

Successfully attacked:	1270
Total attacked: 	1440
Total processed:	1687

Success-Rate: 		0.88
Perturbed Accurracy: 	0.1


{'success_rate': 0.8819444444444444, 'acc': 0.1007705986959099}

In [36]:
attack.showAdversarialExample(target_class=1)